In [ ]:
#This program is a raw version of the cutting program that has all the functions within the file. For a user-friendly approach please refer to WireCutting/main.py
#Most of the testing were done using this file for easier troubleshooting and access to functions. The current setup is for an 8-3 Circuit cut.

In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import re
import pennylane as qml
import numpy as np
import tensorflow as tf
import tf_keras as keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import time
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
tf.keras.backend.set_floatx('float64')

In [ ]:
n_epochs = 50   # Number of optimization epochs
n_layers = 2    # Number of random layers
n_train = 1600    # Size of the train dataset
n_test = 400     # Size of the test dataset

droprate=0.25

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

mnist_dataset = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist_dataset.load_data()
print(X_train.shape)

In [ ]:
X_train=X_train.reshape((len(X_train), -1))
X_test=X_test.reshape((len(X_test), -1))

y_train=y_train.reshape((len(y_train), -1))
y_test=y_test.reshape((len(y_test), -1))

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Reduce dataset size
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_test = X_test[:n_test]
y_test = y_test[:n_test]

from sklearn.preprocessing import normalize
X_train = normalize(X_train)
X_test = normalize(X_test)

In [ ]:
print(X_train[0].shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
import pennylane as qml

n_qubits = 8
n_target_qubits=3
n_inputs = n_qubits
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def uncutcircuit(inputs, weights):
    qml.AngleEmbedding(features=inputs[0], wires=range(n_qubits), rotation='Y')

    for i in range(n_qubits):
        qml.RY(weights[i], wires=i)

    for i in range(n_qubits-1):
        qml.CNOT(wires=[i,i+1])
    qml.CNOT(wires=[n_qubits-1,0])

    for i in range(n_qubits):
        qml.RY(weights[i+n_qubits], wires=i)
    
    for i in range(n_qubits-1):
        qml.CNOT(wires=[i,i+1])
    qml.CNOT(wires=[n_qubits-1,0])
    
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]
    #return qml.expval(qml.PauliZ(0))

In [ ]:
weights = np.random.uniform(high=2 * np.pi, size=(1, 2*n_qubits))
weight_shapes = {"weights": 2*n_qubits}

In [ ]:
quncutlayer = qml.qnn.KerasLayer(uncutcircuit, weight_shapes = weight_shapes, output_dim=n_qubits)

In [ ]:
draw_input=np.random.uniform(high=2 * np.pi, size=(n_qubits))
draw_weights=np.random.uniform(high=2 * np.pi, size=(2*n_qubits))

reshaped_input = draw_input.reshape(1,n_qubits)
draw_input=tuple(reshaped_input)
print(qml.draw(uncutcircuit)(draw_input, draw_weights))

In [ ]:
clayer_1 = tf.keras.layers.Dense(n_qubits)
clayer_100 = tf.keras.layers.Dense(100)
clayer_2 = tf.keras.layers.Dense(10, activation="softmax")
dropout= tf.keras.layers.Dropout(rate=droprate)
uncutmodel = tf.keras.models.Sequential([clayer_1, quncutlayer, clayer_100, dropout, clayer_2])


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
# model.compile(opt, loss="mae", metrics=["accuracy"])

uncutmodel.compile(loss='sparse_categorical_crossentropy',
      optimizer=opt,
      metrics=['accuracy'])

In [ ]:
q_history = uncutmodel.fit(X_train, y_train, epochs=n_epochs, batch_size=1, validation_data=(X_test, y_test), verbose=2)

In [ ]:
plt.style.use("seaborn-v0_8")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 9))

ax1.plot(q_history.history["val_accuracy"], label="Standard")
ax1.set_ylabel("Validation Accuracy")
ax1.set_ylim([0, 1])
ax1.set_xlabel("Epoch")
ax1.legend()

ax2.plot(q_history.history["accuracy"], label="Standard")
ax2.set_ylabel("Training Accuracy")
ax2.set_ylim([0, 1])
ax2.set_xlabel("Epoch")
ax2.legend()
plt.tight_layout()
plt.show()

In [ ]:
uncutmodel.save_weights("uncutmodel.weights.h5")

In [ ]:
def generate_quantum_string(qubits, layers):
    circuit = []
    ry_counter = 1  # Start counting from 1
    for i in range(qubits): #This is neccessary as circuit_to_qasm outputs a set of ry for embedding, this simulates it and is neccessary for the logic of the string_sort
            circuit.append(f"ry({ry_counter}) q[{i}];")

    for layer in range(layers):
        # ry gates with counter-based angle
        for i in range(qubits):
            circuit.append(f"ry({ry_counter}) q[{i}];")
            ry_counter += 1

        # cx gates in a ring
        for i in range(qubits):
            circuit.append(f"cx q[{i}],q[{(i + 1) % qubits}];")

    # Final measurement
    for i in range(qubits):
        circuit.append(f"measure q[{i}] -> c[{i}];")

    return "\n".join(circuit)

In [ ]:
t=generate_quantum_string(n_qubits,n_layers)
print(t)

In [ ]:
patterns = {
    'rxyz': (re.compile(r'r(y|z|x)\((-?\d+(\.\d+)?)\) q\[(\d+)\];'), lambda m: int(m.group(4))),
    'ht-pauli': (re.compile(r'(h|t|x|y|z) q\[(\d+)\];'), lambda m: int(m.group(2))),
    
    'cx': (re.compile(r'cx q\[(\d+)\],q\[(\d+)\];'), lambda m: (int(m.group(1)), int(m.group(2)))),
    'id': (re.compile(r'id q\[(\d+)\],q\[(\d+)\];'), lambda m: ((int(m.group(1)), int(m.group(2))),))
}


func_map = {
    'ry': qml.RY,
    'rx': qml.RX,
    'rz': qml.RZ,
    'cx': qml.CNOT,
    'id': qml.Identity,
    'h': qml.Hadamard,
    'x': qml.PauliX,
    'y': qml.PauliY,
    'z': qml.PauliZ,
    't': qml.T
}

In [ ]:
def string_sort(input_str, num_layers):
    temp_string = ""
    split_str = input_str.splitlines()
    counters=0
    ry_ctr=0

    for i in range(num_layers):
        for string in split_str:
            line = string.strip()
            for key, (pattern, args_func) in patterns.items():
                match = pattern.match(line)
                if match:
                    wire = args_func(match)
                    if isinstance(wire, tuple):  # Check if wire is a tuple
                        wire_list = wire
                        position_not_found = True
                        reverse_string = ""
                        for gate in reversed(temp_string.split("\n")):
                            gate = gate.strip()
                            if gate:
                                for p, af in patterns.values():
                                    gate_match = p.match(gate)
                                    if gate_match:
                                        gate_wire = af(gate_match)
                                        if not isinstance(gate_wire, tuple):
                                            gate_wire = (gate_wire,)  # Convert to tuple if not already
                                        if any(w in wire_list for w in gate_wire) and position_not_found:
                                            reverse_string += line + "\n" + gate + "\n"
                                            position_not_found = False
                                        else:
                                            reverse_string += gate + "\n"
                                        break
                                else:
                                    reverse_string += gate + "\n"
                        if position_not_found:
                            reverse_string += line + "\n"
                        temp_string = "\n".join(reversed(reverse_string.split("\n"))).strip()+"\n"
                        break
                    elif 'r' in line:
                        if ry_ctr>=n_qubits:
                            line = f"r{match.group(1)}({counters}) q[{match.group(4)}];"
                            counters += 1
                            temp_string += line + "\n"
                        else:
                            ry_ctr+=1
                    else:
                        temp_string += string + '\n'
    return temp_string.strip()

In [ ]:
sorted=string_sort(t, 1)
print(sorted)

In [ ]:
def cut_placement(input_str, target_qubits):
    temp_string = ""
    subwires = set()
    split_str = input_str.splitlines()
    subwires_wire_list = []  # Initialize as an empty list
    for string in split_str:
        line = string.strip()
        for pattern, args_func in patterns.values():
            match = pattern.match(line)
            if match:
                wire = args_func(match)
                if isinstance(wire, int):
                    wire = (wire,)
                else:
                    wire = tuple(wire)
                if len(subwires.union(wire)) <= target_qubits:
                    subwires.update(wire)
                    temp_string += string + '\n'
                else:
                    temp_string += "CUT HERE\n" + string + '\n'
                    subwires_wire_list.append(list(subwires))  # Record the subwires before cutting
                    subwires = set(wire)  # Reset subwires to the current wire
    if subwires:  # Append any remaining subwires after the loop
        subwires_wire_list.append(list(subwires))
    return temp_string.strip(), subwires_wire_list

In [ ]:
cut, wire_list=cut_placement(sorted,n_target_qubits)
print(cut+'\n')
print(wire_list)

In [ ]:
def build_sub(input_str, subwires):
    target_qubits=len(subwires)
    n_inputs=len(subwires)
    dev = qml.device('default.qubit', wires=target_qubits)
    @qml.qnode(dev)
    def circuit(inputs, weights):
        qml.AngleEmbedding(features=inputs[0], wires=range(n_qubits), rotation='Y')
        for line in input_str.splitlines():
            line = line.strip()
            for key, (pattern, args_func) in patterns.items():
                match = pattern.match(line)
                if match:
                    args = args_func(match)
                    if isinstance(args, int) and 'r' in line:
                        func_map[f"r{match.group(1)}"](weights[int(match.group(2))], subwires.index(args))  # Do not pass 'inputs' to other function
                    elif not isinstance(args, int):
                        wires=[]
                        for x in args:
                            wires.append(subwires.index(x))
                        func_map[key](wires)
                    else:
                        func_map[match.group(1)](subwires.index(args))
                        
        return [qml.expval(qml.PauliZ(j)) for j in range(target_qubits)]
    return circuit

In [ ]:
def build_model(input_shape, classical_before, neurons_before, activation_before, classical_after, neurons_after, activation_after, qlayer_string, subwires, weight_shapes, weights):
    unique_sets = set(element for sublist in subwires for element in sublist)
    oldqubits = len(unique_sets)
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs

    for i in range(classical_before):
        layer=tf.keras.layers.Dense(neurons_before[i], activation=activation_before[i])
        x=layer(x)


    states = tf.split(x, num_or_size_splits=oldqubits, axis=1)
    states_list = list(states)
    

    subcircuits=qlayer_string.split("CUT HERE")
    draw_input=np.random.uniform(high=2 * np.pi, size=(oldqubits))
    draw_weights=np.random.uniform(high=2 * np.pi, size=(2*oldqubits))
    
    print(draw_input)
    print(draw_weights)
    
    wire_list_ctr = 0
    for str in subcircuits:
        subwire_list=subwires[wire_list_ctr] #wires of that subcircuit
        target_qubits=len(subwire_list)
    
        substate_ctr = 0 #for updating states later
        need_states = [states_list[i] for i in subwire_list] #get states needed for wires of subcircuit
        need_inputs_draw = np.array([draw_input[i] for i in subwire_list])
        reshaped_input = need_inputs_draw.reshape(1,target_qubits)
        need_inputs_draw=tuple(reshaped_input)
        
        # Ensuring the tensor is concatenated correctly
        need_states_list_32=[]
        for state in need_states:
            need_state_32=tf.cast(state, tf.float32)
            need_states_list_32.append(need_state_32)
        print(f"X before shape: {len(need_states_list_32)}")
        print(f"X before type: {type(need_states_list_32)}")
        x = tf.concat(need_states_list_32, axis=1)
        print(f"X shape: {x.shape}")
        print(f"X type: {type(x)}")
        print(f" X: {x}")
        print(f" randdraw: {need_inputs_draw}")

        #build qlayer
        circuit = build_sub(str, subwire_list)
        print(subwire_list)
        print(need_inputs_draw)
        print(qml.draw(circuit)(need_inputs_draw, draw_weights))

        qlayer = qml.qnn.KerasLayer(circuit, weight_shapes, output_dim=target_qubits)

        #run layer and get states
        x = qlayer(x)
        substates = tf.split(x, num_or_size_splits=len(subwire_list), axis=1)
        substates_list = list(substates)
        
        for wire in subwire_list:
            states_list[wire] = substates_list[substate_ctr]
            substate_ctr += 1
        
        wire_list_ctr += 1

    # reassemble states
    states_list_32=[]
    for state in states_list:
        state_32=tf.cast(state, tf.float32)
        states_list_32.append(state_32)
    x = tf.concat(states_list, axis=1)
    
    for i in range(classical_after):
        layer=tf.keras.layers.Dense(neurons_after[i], activation=activation_after[i])
        x=layer(x)
    
    outputs = x
    return tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
cutmodel = build_model((784,),
                       1,
                       [n_qubits],
                       [None],
                       1,
                       [10],
                       ['softmax'],
                       cut,
                       wire_list,
                       weight_shapes,
                       weights)

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

cutmodel.compile(loss='sparse_categorical_crossentropy',
      optimizer=opt,
      metrics=['accuracy'])

In [ ]:
cutmodel.summary()

In [ ]:
cut_start_time=time.time()
q_history = cutmodel.fit(X_train, y_train, epochs=n_epochs, batch_size=1, validation_data=(X_test, y_test), verbose=2)
cut_end_time=time.time()

In [ ]:
cut_runtime=cut_end_time-cut_start_time
print(cut_runtime)

In [ ]:
plt.style.use("seaborn-v0_8")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 9))

ax1.plot(q_history.history["val_accuracy"], label="Standard")
ax1.set_ylabel("Validation Accuracy")
ax1.set_ylim([0, 1])
ax1.set_xlabel("Epoch")
ax1.legend()

ax2.plot(q_history.history["accuracy"], label="Standard")
ax2.set_ylabel("Training Accuracy")
ax2.set_ylim([0, 1])
ax2.set_xlabel("Epoch")
ax2.legend()
plt.tight_layout()
plt.show()

In [ ]:
cutmodel.save_weights("8-3cutmodel_REMOTE.weights.h5")

In [ ]:
import pandas as pd

hist_df = pd.DataFrame(q_history.history)
# or save to csv: 
hist_csv_file = '30-5cut.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)